Устанавливаем и импортируем библиотеки

In [43]:
!pip3 install transformers datasets matplotlib tables wandb

   ---------------------------------------- 0.0/6.5 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.5 MB 4.2 MB/s eta 0:00:02
   ----------- ---------------------------- 1.8/6.5 MB 7.2 MB/s eta 0:00:01
   ---------------------- ----------------- 3.7/6.5 MB 7.5 MB/s eta 0:00:01
   -------------------------------- ------- 5.2/6.5 MB 7.4 MB/s eta 0:00:01
   ---------------------------------------- 6.5/6.5 MB 8.5 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import json

Скачиваем датасет и удаляем ненужные столбцы

In [3]:
data = load_dataset('suriyagunasekar/stackoverflow-python-with-meta-data')
data = data['train']
data = data.to_pandas()

del data['content']
del data['title']
del data['non_answers']
del data['non_answers_scores']
del data['tags']
del data['name']

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/19 [00:00<?, ?it/s]

Удаляем вопросы на котороые нет ответа и вопросы на которые ответ собрал 0 голоов. 

In [4]:
data = data[data['answers_scores'].map(len) > 0]
data = data[data['answers_scores'].map(lambda x: x[0]) > 0]

Удаляем столбец с голосами за ответ и оставляем только первый ответ на вопрос.

In [5]:
del data['answers_scores']

In [6]:
data['answers'] = data['answers'].progress_apply(lambda x: x[0])

100%|███████████████████████████████████████████████████████████████████| 1211172/1211172 [00:00<00:00, 1498639.45it/s]


Добавляем вводный текст для нейросети и создаем готовый для токенизации текст. 

In [7]:
def generate_full_text(line):
    return 'You are an expert who must answer a user’s question on the site regarding questions about python programming. User question: ' + line['question'] + ' Your answer: ' + line['answers']

In [8]:
data['full_text'] = data.progress_apply(generate_full_text, axis = 1)

100%|████████████████████████████████████████████████████████████████████| 1211172/1211172 [00:11<00:00, 106901.23it/s]


In [9]:
del data['question']
del data['answers']

Токенизируем данные, делая это по частям чтобы хватало памяти.

In [62]:
max_length = 900

tokenizer = AutoTokenizer.from_pretrained(
    'Qwen/Qwen1.5-0.5B',
    padding_side="left",
    add_eos_token=True,  
    add_bos_token=True,  
)
tokenizer.pad_token = tokenizer.eos_token


def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [63]:
test = data[:100]

In [64]:
test['full_text'] = test['full_text'].progress_apply(tokenize)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 694.35it/s]
C:\Users\User_PC\AppData\Local\Temp\ipykernel_7240\4140176995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['full_text'] = test['full_text'].progress_apply(tokenize)


In [13]:
test.to_json('test.json', orient = 'records')

In [11]:
data1 = data[:int(len(data) / 16)].reset_index()
data2 = data[int(len(data) / 16) : int(len(data) / 16) * 2].reset_index()
data3 = data[int(len(data) / 16) * 2 : int(len(data) / 16) * 3].reset_index()
data4 = data[int(len(data) / 16) * 3 : int(len(data) / 16) * 4].reset_index()
data5 = data[int(len(data) / 16) * 4 : int(len(data) / 16) * 5].reset_index()
data6 = data[int(len(data) / 16) * 5 : int(len(data) / 16) * 6].reset_index()
data7 = data[int(len(data) / 16) * 6 : int(len(data) / 16) * 7].reset_index()
data8 = data[int(len(data) / 16) * 7 : int(len(data) / 16) * 8].reset_index()
data9 = data[int(len(data) / 16) * 8 : int(len(data) / 16) * 9].reset_index()
data10 = data[int(len(data) / 16) * 9 : int(len(data) / 16) * 10].reset_index()
data11 = data[int(len(data) / 16) * 10 : int(len(data) / 16) * 11].reset_index()
data12 = data[int(len(data) / 16) * 11 : int(len(data) / 16) * 12].reset_index()
data13 = data[int(len(data) / 16) * 12 : int(len(data) / 16) * 13].reset_index()
data14 = data[int(len(data) / 16) * 13 : int(len(data) / 16) * 14].reset_index()
data15 = data[int(len(data) / 16) * 14 : int(len(data) / 16) * 15].reset_index()
data16 = data[int(len(data) / 16) * 15 :].reset_index()

del data1['index']
del data2['index']
del data3['index']
del data4['index']
del data5['index']
del data6['index']
del data7['index']
del data8['index']
del data9['index']
del data10['index']

In [12]:
data1['full_text'] = data1['full_text'].progress_apply(tokenize)

100%|███████████████████████████████████████████████████████████████████████████| 75698/75698 [01:27<00:00, 866.34it/s]


In [13]:
data1.to_json('data1.json', orient = 'records')

In [ ]:
data2['full_text'] = data2['full_text'].progress_apply(tokenize)

In [23]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen1.5-0.5B')

In [45]:
import wandb, os
wandb.login()

wandb_project = "qwen-finetune"
os.environ["WANDB_PROJECT"] = wandb_project

wandb: Currently logged in as: n-d-arkhipov (n-d-arkhipov-home). Use `wandb login --relogin` to force relogin


In [65]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type = TaskType.CAUSAL_LM, r = 8, lora_alpha = 16, lora_dropout = 0.1)
model = get_peft_model(model, config)

In [66]:
model.print_trainable_parameters()

trainable params: 786,432 || all params: 464,774,144 || trainable%: 0.1692


In [67]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(output_dir = 'D:\SO_python', learning_rate=2.5e-5, per_device_train_batch_size=2, num_train_epochs = 2, eval_strategy="epoch", save_strategy="epoch",report_to="wandb")

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
trainer = Trainer(model = model, args=training_args, train_dataset = test['full_text'], tokenizer=tokenizer, data_collator = data_collator)

In [ ]:
trainer.train()